# CTHD August Sales Analysis

## Preface

## Data Cleaning

## Data Exploratory

## Visualization

# Data Cleaning

In [1]:
# Import the modules
import pandas as pd
import numpy as np

In [2]:
# the main data we are gonna use
# Original csv file that Ian provided: 臥虎藏龍 Crouch & Hide _ Tiger Dragon Bar月報.csv
# renamed it to cthd_report_aug.csv

file_path = "/Users/lucaslee/Documents/GitHub/Lucas-Data-Scientist-Lab/data_cthd_Aug/cthd_report_aug.csv"


In [3]:
df_raw = pd.read_csv(file_path, skiprows=1)

# print original columns:
print(df_raw.columns)
# result: ['銷售日期', '星期', '銷售總額', '折扣金額', '營業額', '稅額', '營業淨額', '客數', '周轉率', '人均價', '交易數', '交易均價']

Index(['銷售日期', '星期', '銷售總額', '折扣金額', '營業額', '稅額', '營業淨額', '客數', '周轉率', '人均價',
       '交易數', '交易均價'],
      dtype='object')


In [4]:
# Only select the rows with aug info in:
df = df_raw.loc[df_raw['銷售日期'].str.contains('2022-08')]

# Select columns that we are interested
df = df[[
    '銷售日期', '星期', '銷售總額', '折扣金額', '營業額', '客數', '人均價', '交易數', '交易均價'
]]

# Rename the column to English
df.columns = [
    'sales_date', 'weekday', 'sales_total', 'sales_discount', 'sales_result', 'guest_num', 'price_per_guest', 'orders_num', 'price_per_order'
]

In [5]:
df['sales_total'] = df['sales_total'].str.replace(",", "").astype(int)
df['sales_discount'] = df['sales_discount'].str.replace(",", "").astype(int)
df['sales_result'] = df['sales_result'].str.replace(",", "").astype(int)
df['guest_num'] = df['guest_num'].str.replace(",", "").astype(int)
df['price_per_guest'] = df['price_per_guest'].str.replace(",", "").astype(int)
df['price_per_order'] = df['price_per_order'].str.replace(",", "").astype(int)

In [6]:
# weekday transform: using np.select
condlist = [
    df['weekday'] == '一',
    df['weekday'] == '二',
    df['weekday'] == '三',
    df['weekday'] == '四',
    df['weekday'] == '五',
    df['weekday'] == '六',
    df['weekday'] == '日',
]
choicelist = [
    'Mon',
    'Tue',
    'Wed',
    'Thu',
    'Fri',
    'Sat',
    'Sun'
]

df['weekday'] = np.select(condlist, choicelist, 'Unknown')


In [7]:
# sales_date transform to datetime
df['sales_date'] = pd.to_datetime(df.sales_date)

In [8]:
# weekday vs weekend
weekday_list = [
    'Mon',
    'Tue',
    'Wed',
    'Thu',
]
weekend_list = [
    'Fri', 'Sat', 'Sun'
]

# Using np.select()
condlist = [df['weekday'].isin(weekday_list), df['weekday'].isin(weekend_list)]
choicelist = ['weekday', 'weekend']
df['weekend_label'] = np.select(condlist, choicelist, 'Unknown')


df.pivot_table(index=['weekend_label'],
               values=['sales_result', 'sales_discount', 'guest_num'],
               aggfunc='mean')

,guest_num,sales_discount,sales_result
weekend_label,,,
weekday,29.473684,434.894737,28873.736842
weekend,62.333333,929.916667,64353.250000


In [10]:
df.to_csv('cthd_report_aug_transform.csv', index=False)